### N-gram language models or how to write scientific papers (4 pts)

We shall train our language model on a corpora of [ArXiv](http://arxiv.org/) articles and see if we can generate a new one!

![img](https://media.npr.org/assets/img/2013/12/10/istock-18586699-monkey-computer_brick-16e5064d3378a14e0e4c2da08857efe03c04695e-s800-c85.jpg)

_data by neelshah18 from [here](https://www.kaggle.com/neelshah18/arxivdataset/)_

_Disclaimer: this has nothing to do with actual science. But it's fun, so who cares?!_

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
# Alternative manual download link: https://yadi.sk/d/_nGyU2IajjR9-w
# !wget "https://www.dropbox.com/s/99az9n1b57qkd9j/arxivData.json.tar.gz?dl=1" -O arxivData.json.tar.gz
# !tar -xvzf arxivData.json.tar.gz
data = pd.read_json("./resources/arxivData.json")
data.sample(n=5)

,author,day,id,link,month,summary,tag,title,year
12230,"[{'name': 'Yunchuan Kong'}, {'name': 'Xiaodan ...",24,1710.08846v1,"[{'rel': 'alternate', 'href': 'http://arxiv.or...",10,We present a new model-based integrative metho...,"[{'term': 'stat.ML', 'scheme': 'http://arxiv.o...",A Bayesian Method for Joint Clustering of Vect...,2017
7124,"[{'name': 'Amal Rannen Triki'}, {'name': 'Maxi...",18,1710.06703v1,"[{'rel': 'alternate', 'href': 'http://arxiv.or...",10,Deep neural networks (DNNs) have become increa...,"[{'term': 'cs.LG', 'scheme': 'http://arxiv.org...",Stochastic Weighted Function Norm Regularization,2017
26646,"[{'name': 'Junbo Wang'}, {'name': 'Wei Wang'},...",17,1611.05592v1,"[{'rel': 'alternate', 'href': 'http://arxiv.or...",11,Video captioning which automatically translate...,"[{'term': 'cs.CV', 'scheme': 'http://arxiv.org...",Multimodal Memory Modelling for Video Captioning,2016
33543,[{'name': 'Boulif Menouar'}],16,1612.05536v1,"[{'rel': 'alternate', 'href': 'http://arxiv.or...",12,Cell formation is a critical step in the desig...,"[{'term': 'cs.DM', 'scheme': 'http://arxiv.org...",A new cut-based genetic algorithm for graph pa...,2016
29661,"[{'name': 'Helen L. Bear'}, {'name': 'Gari Owe...",3,1710.01084v1,"[{'rel': 'alternate', 'href': 'http://arxiv.or...",10,In the quest for greater computer lip-reading ...,"[{'term': 'cs.CV', 'scheme': 'http://arxiv.org...",Some observations on computer lip-reading: mov...,2017


In [7]:
data.summary.values[:2]

array(['We propose an architecture for VQA which utilizes recurrent layers to\ngenerate visual and textual attention. The memory characteristic of the\nproposed recurrent attention units offers a rich joint embedding of visual and\ntextual features and enables the model to reason relations between several\nparts of the image and question. Our single model outperforms the first place\nwinner on the VQA 1.0 dataset, performs within margin to the current\nstate-of-the-art ensemble model. We also experiment with replacing attention\nmechanisms in other state-of-the-art models with our implementation and show\nincreased accuracy. In both cases, our recurrent attention mechanism improves\nperformance in tasks requiring sequential or relational reasoning on the VQA\ndataset.',
       'Recent approaches based on artificial neural networks (ANNs) have shown\npromising results for short-text classification. However, many short texts\noccur in sequences (e.g., sentences in a document or utterance

In [11]:
# assemble lines: concatenate title and description
lines = data.apply(lambda row: row['title'] + ' ; ' + row['summary'].replace('\n', ' '), axis=1).tolist()

sorted(lines, key=len)[:3]

['Differential Contrastive Divergence ; This paper has been retracted.',
 'What Does Artificial Life Tell Us About Death? ; Short philosophical essay',
 'P=NP ; We claim to resolve the P=?NP problem via a formal argument for P=NP.']

# Tokenization

You know the dril. The data is messy. Go clean the data. Use WordPunctTokenizer or something.


In [12]:
import nltk
tokenizer = nltk.tokenize.WordPunctTokenizer()

In [17]:
# Task: convert lines (in-place) into strings of space-separated tokens. import & use WordPunctTokenizer

lines = [' '.join(tokenizer.tokenize(line.lower())) for line in lines]

In [19]:
assert sorted(lines, key=len)[0] == \
    'differential contrastive divergence ; this paper has been retracted .'
assert sorted(lines, key=len)[2] == \
    'p = np ; we claim to resolve the p =? np problem via a formal argument for p = np .'

### N-Gram Language Model (1point)

A language model is a probabilistic model that estimates text probability: the joint probability of all tokens $w_t$ in text $X$: $P(X) = P(w_1, \dots, w_T)$.

It can do so by following the chain rule:
$$P(w_1, \dots, w_T) = P(w_1)P(w_2 \mid w_1)\dots P(w_T \mid w_1, \dots, w_{T-1}).$$ 

The problem with such approach is that the final term $P(w_T \mid w_1, \dots, w_{T-1})$ depends on $n-1$ previous words. This probability is impractical to estimate for long texts, e.g. $T = 1000$.

One popular approximation is to assume that next word only depends on a finite amount of previous words:

$$P(w_t \mid w_1, \dots, w_{t - 1}) = P(w_t \mid w_{t - n + 1}, \dots, w_{t - 1})$$

Such model is called __n-gram language model__ where n is a parameter. For example, in 3-gram language model, each word only depends on 2 previous words. 

$$
    P(w_1, \dots, w_n) = \prod_t P(w_t \mid w_{t - n + 1}, \dots, w_{t - 1}).
$$

You can also sometimes see such approximation under the name of _n-th order markov assumption_.

The first stage to building such a model is counting all word occurences given N-1 previous words

In [202]:
a=[i for i in range(10)]
n=4
for i in range(n-1,len(a)):
    print(tuple(a[i-(n-1):i]))

(0, 1, 2)
(1, 2, 3)
(2, 3, 4)
(3, 4, 5)
(4, 5, 6)
(5, 6, 7)
(6, 7, 8)


In [203]:
from tqdm import tqdm
from collections import defaultdict, Counter

# special tokens: 
# - unk represents absent tokens, 
# - eos is a special token after the end of sequence

UNK, EOS = "_UNK_", "_EOS_"

def count_ngrams(lines, n):
    """
    Count how many times each word occured after (n - 1) previous words
    :param lines: an iterable of strings with space-separated tokens
    :returns: a dictionary { tuple(prefix_tokens): {next_token_1: count_1, next_token_2: count_2}}

    When building counts, please consider the following two edge cases
    - if prefix is shorter than (n - 1) tokens, it should be padded with UNK. For n=3,
      empty prefix: "" -> (UNK, UNK)
      short prefix: "the" -> (UNK, the)
      long prefix: "the new approach" -> (new, approach)
    - you should add a special token, EOS, at the end of each sequence
      "... with deep neural networks ." -> (..., with, deep, neural, networks, ., EOS)
      count the probability of this token just like all others.
    """
    counts = defaultdict(Counter)
    
    for line in lines:
        tokens = [UNK]*(n-1) +line.split(' ') + [EOS]
        for i in range(n-1,len(tokens)):
            counts[tuple(tokens[i-(n-1):i])][tokens[i]]+=1
#             counts[(tokens[i-2],tokens[i-1])][tokens[i]]+=1
    # counts[(word1, word2)][word3] = how many times word3 occured after (word1, word2)    
    return counts


In [204]:
# let's test it
dummy_lines = sorted(lines, key=len)[:100]
dummy_counts = count_ngrams(dummy_lines, n=3)
assert set(map(len, dummy_counts.keys())) == {2}, "please only count {n-1}-grams"
assert len(dummy_counts[('_UNK_', '_UNK_')]) == 78
assert dummy_counts['_UNK_', 'a']['note'] == 3
assert dummy_counts['p', '=']['np'] == 2
assert dummy_counts['author', '.']['_EOS_'] == 1

Once we can count N-grams, we can build a probabilistic language model.
The simplest way to compute probabilities is in proporiton to counts:

$$ P(w_t | prefix) = { Count(prefix, w_t) \over \sum_{\hat w} Count(prefix, \hat w) } $$

In [34]:
a = {'b':1,'c':3}
for i in a:
    a[i]**=2
a

{'b': 1, 'c': 9}

In [216]:
class NGramLanguageModel:    
    def __init__(self, lines, n):
        """ 
        Train a simple count-based language model: 
        compute probabilities P(w_t | prefix) given ngram counts
        
        :param n: computes probability of next token given (n - 1) previous words
        :param lines: an iterable of strings with space-separated tokens
        """
        assert n >= 1
        self.n = n
    
        counts = count_ngrams(lines, self.n)
        
        # compute token proabilities given counts
        self.probs = defaultdict(dict)
        # probs[(word1, word2)][word3] = P(word3 | word1, word2)
        
        # populate self.probs with actual probabilities
        for prefix in counts.keys():
            for word in counts[prefix].keys():
                self.probs[prefix][word] = counts[prefix][word] / sum(counts[prefix].values())
        
#         for word1, word2 in counts.keys():
#             for word3 in counts[(word1, word2)].keys():
#                 self.probs[(word1, word2)][word3] = counts[(word1, word2)][word3]/ sum(counts[(word1, word2)].values())
            
    def get_possible_next_tokens(self, prefix):
        """
        :param prefix: string with space-separated prefix tokens
        :returns: a dictionary {token : it's probability} for all tokens with positive probabilities
        """
        prefix = prefix.split()
        prefix = prefix[max(0, len(prefix) - self.n + 1):]
        prefix = [ UNK ] * (self.n - 1 - len(prefix)) + prefix
        return self.probs[tuple(prefix)]
    
    def get_next_token_prob(self, prefix, next_token):
        """
        :param prefix: string with space-separated prefix tokens
        :param next_token: the next token to predict probability for
        :returns: P(next_token|prefix) a single number, 0 <= P <= 1
        """
        return self.get_possible_next_tokens(prefix).get(next_token, 0)

Let's test it!

In [217]:
dummy_lm = NGramLanguageModel(dummy_lines, n=3)

p_initial = dummy_lm.get_possible_next_tokens('') # '' -> ['_UNK_', '_UNK_']
assert np.allclose(p_initial['learning'], 0.02)
assert np.allclose(p_initial['a'], 0.13)
assert np.allclose(p_initial.get('meow', 0), 0)
assert np.allclose(sum(p_initial.values()), 1)

p_a = dummy_lm.get_possible_next_tokens('a') # '' -> ['_UNK_', 'a']
assert np.allclose(p_a['machine'], 0.15384615)
assert np.allclose(p_a['note'], 0.23076923)
assert np.allclose(p_a.get('the', 0), 0)
assert np.allclose(sum(p_a.values()), 1)

assert np.allclose(dummy_lm.get_possible_next_tokens('a note')['on'], 1)
assert dummy_lm.get_possible_next_tokens('a machine') == \
    dummy_lm.get_possible_next_tokens("there have always been ghosts in a machine"), \
    "your 3-gram model should only depend on 2 previous words"

Now that you've got a working n-gram language model, let's see what sequences it can generate. But first, let's train it on the whole dataset.

In [110]:
lm = NGramLanguageModel(lines, n=3)

The process of generating sequences is... well, it's sequential. You maintain a list of tokens and iteratively add next token by sampling with probabilities.

$ X = [] $

__forever:__
* $w_{next} \sim P(w_{next} | X)$
* $X = concat(X, w_{next})$


Instead of sampling with probabilities, one can also try always taking most likely token, sampling among top-K most likely tokens or sampling with temperature. In the latter case (temperature), one samples from

$$w_{next} \sim {P(w_{next} | X) ^ {1 / \tau} \over \sum_{\hat w} P(\hat w | X) ^ {1 / \tau}}$$

Where $\tau > 0$ is model temperature. If $\tau << 1$, more likely tokens will be sampled with even higher probability while less likely tokens will vanish.

In [218]:
def get_next_token(lm, prefix, temperature=1.0):
    """
    return next token after prefix;
    :param temperature: samples proportionally to lm probabilities ^ (1 / temperature)
        if temperature == 0, always takes most likely token. Break ties arbitrarily.
    """
    # Get possible next tokens
    # next_tokens -> dict{token1:p1, token2:p2...}
    next_token_probability = lm.get_possible_next_tokens(prefix).copy()
    if temperature==0:
        # Return the most likely token
        return sorted(next_token_probability,key=next_token_probability.get,reverse=True)[0]
    else:
        for key in next_token_probability:
            next_token_probability[key] **=(1/temperature)
            
        new_sum = sum(next_token_probability.values())
        
        for key in next_token_probability:
            next_token_probability[key] /=new_sum
            
        return list(next_token_probability.keys())[np.random.choice(len(next_token_probability), 1, p=list(next_token_probability.values()))[0]]

    

In [208]:
temperature=2
test = lm.get_possible_next_tokens('there have').copy()
print(test)

for key in test:
    test[key] **=(1/temperature)

new_sum = sum(test.values())

for key in test:
    test[key] /=new_sum
print(test)

{'been': 0.9083969465648855, 'not': 0.03816793893129771, 'only': 0.015267175572519083, 'also': 0.015267175572519083, 'lately': 0.007633587786259542, 'very': 0.007633587786259542, 'occurred': 0.007633587786259542}
{'been': 0.5749535115459828, 'not': 0.11785397966402933, 'only': 0.07453740141069969, 'also': 0.07453740141069969, 'lately': 0.05270590198952948, 'very': 0.05270590198952948, 'occurred': 0.05270590198952948}


In [101]:
test_freqs = Counter([get_next_token(lm, 'there have') for _ in range(10000)])
test_freqs

Counter({'been': 9049,
         'very': 75,
         'only': 175,
         'not': 399,
         'lately': 86,
         'also': 142,
         'occurred': 74})

In [209]:
from collections import Counter
test_freqs = Counter([get_next_token(lm, 'there have') for _ in range(10000)])
assert 250 < test_freqs['not'] < 450
assert 8500 < test_freqs['been'] < 9500
assert 1 < test_freqs['lately'] < 200

test_freqs = Counter([get_next_token(lm, 'deep', temperature=1.0) for _ in range(10000)])
assert 1500 < test_freqs['learning'] < 3000
test_freqs = Counter([get_next_token(lm, 'deep', temperature=0.5) for _ in range(10000)])
assert 8000 < test_freqs['learning'] < 9000
test_freqs = Counter([get_next_token(lm, 'deep', temperature=0.0) for _ in range(10000)])
assert test_freqs['learning'] == 10000

print("Looks nice!")

Looks nice!


Let's have fun with this model

In [118]:
prefix = 'artificial' # <- your ideas :)

for i in range(100):
    prefix += ' ' + get_next_token(lm, prefix)
    if prefix.endswith(EOS) or len(lm.get_possible_next_tokens(prefix)) == 0:
        break
        
print(prefix)

artificial neural networks ) would lead to the baseline , while preserving their identity has been whether dl - lite , that collaboratively contribute to belief change ; languages for which they can specify the relations among features from the rest is based on bayesian networks . this method can also achieve competitive accuracy . extensive experiments on various benchmarks and achieve a base transceiver station ( bts ) broadcasts multiple data ( e . g . classification of demographic dynamics within it to provide high - accuracy . _EOS_


In [119]:
prefix = 'bridging the' # <- more of your ideas

for i in range(100):
    prefix += ' ' + get_next_token(lm, prefix, temperature=0.5)
    if prefix.endswith(EOS) or len(lm.get_possible_next_tokens(prefix)) == 0:
        break
        
print(prefix)

bridging the gap between the two tasks : ( i . e . g ., the number of layers , and the algorithm . we also propose a probabilistic model for the purpose of this paper , we propose a novel and robust method even outperforms it for learning . _EOS_


In [128]:
prefix = 'what i can say' # <- more of your ideas

for i in range(100):
    prefix += ' ' + get_next_token(lm, prefix, temperature=0.5)
    if prefix.endswith(EOS) or len(lm.get_possible_next_tokens(prefix)) == 0:
        break
        
print(prefix)

what i can say that there are three - dimensional data and real - time and space complexities at once . _EOS_


__More in the homework:__ nucleous sampling, top-k sampling, beam search(not for the faint of heart).

### Evaluating language models: perplexity (1point)

Perplexity is a measure of how well does your model approximate true probability distribution behind data. __Smaller perplexity = better model__.

To compute perplexity on one sentence, use:
$$
    {\mathbb{P}}(w_1 \dots w_N) = P(w_1, \dots, w_N)^{-\frac1N} = \left( \prod_t P(w_t \mid w_{t - n}, \dots, w_{t - 1})\right)^{-\frac1N},
$$


On the corpora level, perplexity is a product of probabilities of all tokens in all sentences to the power of 1, divided by __total length of all sentences__ in corpora.

This number can quickly get too small for float32/float64 precision, so we recommend you to first compute log-perplexity (from log-probabilities) and then take the exponent.

In [131]:
np.log(10 ** -50.)

-115.12925464970229

In [264]:
def perplexity(lm, lines, min_logprob=np.log(10 ** -50.)):
    """
    :param lines: a list of strings with space-separated tokens
    :param min_logprob: if log(P(w | ...)) is smaller than min_logprop, set it equal to min_logrob
    :returns: corpora-level perplexity - a single scalar number from the formula above
    
    Note: do not forget to compute P(w_first | empty) and P(eos | full_sequence)
    
    PLEASE USE lm.get_next_token_prob and NOT lm.get_possible_next_tokens
    """
    UNK, EOS = "_UNK_", "_EOS_"
    corpora_length=0
    log_lines_perplexity = []
    # Walk through each line
    for line in lines:
        tokens = line.split(' ') + [EOS]
        log_line_perplexity=0
        N=0
        # Walk through each token
        for i in range(len(tokens)):
            log_line_perplexity += max(np.log(lm.get_next_token_prob(prefix=' '.join(tokens[:i]), next_token=tokens[i])), min_logprob)
            N+=1
            
        corpora_length+=N
#         print(log_line_perplexity,N,log_line_perplexity*(-1/N))
        log_lines_perplexity.append(log_line_perplexity)
    
    return  np.exp(sum(log_lines_perplexity) / -corpora_length)


In [265]:
lm1 = NGramLanguageModel(dummy_lines, n=1)
ppx1 = perplexity(lm1, dummy_lines[:10])
ppx1

307.24502881156855

In [266]:
lm1 = NGramLanguageModel(dummy_lines, n=1)
lm3 = NGramLanguageModel(dummy_lines, n=3)
lm10 = NGramLanguageModel(dummy_lines, n=10)

ppx1 = perplexity(lm1, dummy_lines)
ppx3 = perplexity(lm3, dummy_lines)
ppx10 = perplexity(lm10, dummy_lines)
ppx_missing = perplexity(lm3, ['the jabberwock , with eyes of flame , '])  # thanks, L. Carrol

print("Perplexities: ppx1=%.3f ppx3=%.3f ppx10=%.3f" % (ppx1, ppx3, ppx10))

assert all(0 < ppx < 500 for ppx in (ppx1, ppx3, ppx10)), "perplexity should be nonnegative and reasonably small"
assert ppx1 > ppx3 > ppx10, "higher N models should overfit and "
assert np.isfinite(ppx_missing) and ppx_missing > 10 ** 6, "missing words should have large but finite perplexity. " \
    " Make sure you use min_logprob right"
assert np.allclose([ppx1, ppx3, ppx10], (318.2132342216302, 1.5199996213739575, 1.1838145037901249))

Perplexities: ppx1=318.213 ppx3=1.520 ppx10=1.184


<ipython-input-264-91263a57f5af>:21: RuntimeWarning: divide by zero encountered in log
  log_line_perplexity += max(np.log(lm.get_next_token_prob(prefix=' '.join(tokens[:i]), next_token=tokens[i])), min_logprob)


Now let's measure the actual perplexity: we'll split the data into train and test and score model on test data only.

In [267]:
from sklearn.model_selection import train_test_split
train_lines, test_lines = train_test_split(lines, test_size=0.25, random_state=42)

for n in (1, 2, 3):
    lm = NGramLanguageModel(n=n, lines=train_lines)
    ppx = perplexity(lm, test_lines)
    print("N = %i, Perplexity = %.5f" % (n, ppx))


<ipython-input-264-91263a57f5af>:21: RuntimeWarning: divide by zero encountered in log
  log_line_perplexity += max(np.log(lm.get_next_token_prob(prefix=' '.join(tokens[:i]), next_token=tokens[i])), min_logprob)


N = 1, Perplexity = 1832.23136
N = 2, Perplexity = 85653987.28543
N = 3, Perplexity = 61999196239911532363776.00000


In [ ]:
# whoops, it just blew up :)

### LM Smoothing

The problem with our simple language model is that whenever it encounters an n-gram it has never seen before, it assigns it with the probabilitiy of 0. Every time this happens, perplexity explodes.

To battle this issue, there's a technique called __smoothing__. The core idea is to modify counts in a way that prevents probabilities from getting too low. The simplest algorithm here is Additive smoothing (aka [Lapace smoothing](https://en.wikipedia.org/wiki/Additive_smoothing)):

$$ P(w_t | prefix) = { Count(prefix, w_t) + \delta \over \sum_{\hat w} (Count(prefix, \hat w) + \delta) } $$

If counts for a given prefix are low, additive smoothing will adjust probabilities to a more uniform distribution. Not that the summation in the denominator goes over _all words in the vocabulary_.

Here's an example code we've implemented for you:

In [268]:
class LaplaceLanguageModel(NGramLanguageModel): 
    """ this code is an example, no need to change anything """
    def __init__(self, lines, n, delta=1.0):
        self.n = n
        counts = count_ngrams(lines, self.n)
        self.vocab = set(token for token_counts in counts.values() for token in token_counts)
        self.probs = defaultdict(Counter)

        for prefix in counts:
            token_counts = counts[prefix]
            total_count = sum(token_counts.values()) + delta * len(self.vocab)
            self.probs[prefix] = {token: (token_counts[token] + delta) / total_count
                                          for token in token_counts}
    def get_possible_next_tokens(self, prefix):
        token_probs = super().get_possible_next_tokens(prefix)
        missing_prob_total = 1.0 - sum(token_probs.values())
        missing_prob = missing_prob_total / max(1, len(self.vocab) - len(token_probs))
        return {token: token_probs.get(token, missing_prob) for token in self.vocab}
    
    def get_next_token_prob(self, prefix, next_token):
        token_probs = super().get_possible_next_tokens(prefix)
        if next_token in token_probs:
            return token_probs[next_token]
        else:
            missing_prob_total = 1.0 - sum(token_probs.values())
            missing_prob_total = max(0, missing_prob_total) # prevent rounding errors
            return missing_prob_total / max(1, len(self.vocab) - len(token_probs))
        

**Disclaimer**: the implementation above assumes all words unknown within a given context to be equally likely, *as well as the words outside of vocabulary*. Therefore, its' perplexity will be lower than it should when encountering such words. Therefore, comparing it with a model with less unknown words will not be fair. When implementing your own smoothing, you may handle this by adding a virtual `UNK` token of non-zero probability. Technically, this will result in a model where probabilities do not add up to $1$, but it is close enough for a practice excercise.

In [269]:
#test that it's a valid probability model
for n in (1, 2, 3):
    dummy_lm = LaplaceLanguageModel(dummy_lines, n=n)
    assert np.allclose(sum([dummy_lm.get_next_token_prob('a', w_i) for w_i in dummy_lm.vocab]), 1), "I told you not to break anything! :)"

In [270]:
for n in (1, 2, 3):
    lm = LaplaceLanguageModel(train_lines, n=n, delta=0.1)
    ppx = perplexity(lm, test_lines)
    print("N = %i, Perplexity = %.5f" % (n, ppx))

N = 1, Perplexity = 977.67559
N = 2, Perplexity = 470.48021
N = 3, Perplexity = 3679.44765


In [ ]:
# optional: try to sample tokens from such a model

# NOT FINISH YET!!!!!!!!!!!!!!!!!!!!!

### Kneser-Ney smoothing (2 points)

Additive smoothing is simple, reasonably good but definitely not a State of The Art algorithm.


Your final task in this notebook is to implement [Kneser-Ney](https://en.wikipedia.org/wiki/Kneser%E2%80%93Ney_smoothing) smoothing.

It can be computed recurrently, for n>1:

$$P_{kn}(w_t | prefix_{n-1}) = { \max(0, Count(prefix_{n-1}, w_t) - \delta) \over \sum_{\hat w} Count(prefix_{n-1}, \hat w)} + \lambda_{prefix_{n-1}} \cdot P_{kn}(w_t | prefix_{n-2})$$

where
- $prefix_{n-1}$ is a tuple of {n-1} previous tokens
- $lambda_{prefix_{n-1}}$ is a normalization constant chosen so that probabilities add up to 1
- Unigram $P_{kn}(w_t | prefix_{n-2})$ corresponds to Kneser Ney smoothing for {N-1}-gram language model.
- Unigram $P_{kn}(w_t)$ is a special case: how likely it is to see x_t in an unfamiliar context

See lecture slides or wiki for more detailed formulae.

__Your task__ is to
- implement KneserNeyLanguageModel
- test it on 1-3 gram language models
- find optimal (within reason) smoothing delta for 3-gram language model with Kneser-Ney smoothing

In [435]:
class KneserNeyLanguageModel(NGramLanguageModel): 
    """ A template for Kneser-Ney language model. Default delta may be suboptimal. """
    def __init__(self, lines, n, delta=1.0):
        self.n = n
        counts = count_ngrams(lines, self.n)
        self.vocab = set(token for token_counts in counts.values() for token in token_counts)
        # compute token proabilities given counts
        self.probs = defaultdict(dict)
        
#         # populate self.probs with actual probabilities
#         for prefix in counts.keys():
#             for word in counts[prefix].keys():
#                 self.probs[prefix][word] = counts[prefix][word] / sum(counts[prefix].values())
# -----  
        self.counts_ = {}
        self.lambda_ = {}
        self.prob_kn_ = defaultdict(dict)
        for i in range(1, n+1):
            self.counts_[f"counts_{i}"] = count_ngrams(lines, i)
            if i == 1:
                total_count = sum(self.counts_[f"counts_{i}"][()].values())
                p_wt = { k:v/total_count for k,v in self.counts_[f"counts_{i}"][()].items()}
                
                self.prob_kn_[1] = {():p_wt}
            else:
                self.lambda_[i-1] = {}
                for prefix in self.counts_[f"counts_{i}"].keys():
                    total_sum= total_count = 0
                    for token in self.counts_[f"counts_{i}"][prefix].keys():
                        total_count += self.counts_[f"counts_{i}"][prefix][token]
                        total_sum += max(0, self.counts_[f"counts_{i}"][prefix][token] - delta)
                        self.lambda_[i-1][prefix] = total_count-total_sum
                
                self.prob_kn_[i]=defaultdict(dict)

                for prefix in self.counts_[f"counts_{i}"].keys():
                    self.prob_kn_[i][prefix]=defaultdict(dict)
                    for word in self.counts_[f"counts_{i}"][prefix].keys():
                        
                        if i == 2:
#                             print(prefix,self.counts_[f"counts_{i}"][prefix][token])
                            if prefix not in p_wt:
                                self.prob_kn_[i][prefix][word] = (max(0, self.counts_[f"counts_{i}"][prefix][word] - delta) / total_count)
                            else:
                                self.prob_kn_[i][prefix][word] = (max(0, self.counts_[f"counts_{i}"][prefix][word] - delta) / total_count) + self.lambda_[i-1][prefix]*p_wt[prefix]
                        else:
                            self.prob_kn_[i][prefix][word] = (max(0, self.counts_[f"counts_{i}"][prefix][word] - delta) / total_count) + self.lambda_[i-1][prefix]*self.prob_kn_[i-1][prefix[1:]][word]
    
        self.probs = self.prob_kn_[n]

    def get_possible_next_tokens(self, prefix):
        """
        :param prefix: string with space-separated prefix tokens
        :returns: a dictionary {token : it's probability} for all tokens with positive probabilities
        """
        prefix = prefix.split()
        prefix = prefix[max(0, len(prefix) - self.n + 1):]
        prefix = [ UNK ] * (self.n - 1 - len(prefix)) + prefix
        return self.probs[tuple(prefix)]
    
    def get_next_token_prob(self, prefix, next_token):
        """
        :param prefix: string with space-separated prefix tokens
        :param next_token: the next token to predict probability for
        :returns: P(next_token|prefix) a single number, 0 <= P <= 1
        """
        return self.get_possible_next_tokens(prefix).get(next_token, 0)

In [428]:
dummy_lm.probs

dict_keys([('_UNK_',), ('differential',), ('contrastive',), ('divergence',), (';',), ('this',), ('paper',), ('has',), ('been',), ('retracted',), ('.',), ('what',), ('does',), ('artificial',), ('life',), ('tell',), ('us',), ('about',), ('death',), ('?',), ('short',), ('philosophical',), ('essay',), ('p',), ('=',), ('np',), ('we',), ('claim',), ('to',), ('resolve',), ('the',), ('=?',), ('problem',), ('via',), ('a',), ('formal',), ('argument',), ('for',), ('computational',), ('geometry',), ('column',), ('38',), ('recent',), ('results',), ('on',), ('curve',), ('reconstruction',), ('are',), ('described',), ('weak',), ('evolvability',), ('equals',), ('strong',), ('an',), ('updated',), ('version',), ('will',), ('be',), ('uploaded',), ('later',), ('creating',), ('new',), ('ontology',), (':',), ('modular',), ('approach',), ('defeasible',), ('reasoning',), ('in',), ('oscar',), ('is',), ('system',), ('description',), ('reasoner',), ('essence',), ("'",), ('of',), ('language',), ('as',), ('used',),

In [426]:
dummy_lm.counts_['counts_2']

defaultdict(collections.Counter,
            {('_UNK_',): Counter({'differential': 1,
                      'what': 1,
                      'p': 1,
                      'computational': 1,
                      'weak': 1,
                      'creating': 1,
                      'defeasible': 1,
                      'essence': 1,
                      'deep': 1,
                      'statistical': 1,
                      'complex': 1,
                      'serious': 1,
                      'preprocessing': 1,
                      'liquid': 1,
                      'mining': 1,
                      'towards': 1,
                      'a': 13,
                      'icon': 1,
                      'recognition': 1,
                      'glottochronologic': 1,
                      'the': 3,
                      'utility': 1,
                      'temporized': 1,
                      'backpropagation': 1,
                      'random': 1,
                      'network': 1,

In [438]:
dummy_lm.lambda_

{1: {('_UNK_',): 78.0,
  ('differential',): 1,
  ('contrastive',): 1,
  ('divergence',): 1,
  (';',): 28.0,
  ('this',): 17.0,
  ('paper',): 6.0,
  ('has',): 1.0,
  ('been',): 2.0,
  ('retracted',): 1,
  ('.',): 19.0,
  ('what',): 1,
  ('does',): 2,
  ('artificial',): 3.0,
  ('life',): 2,
  ('tell',): 1,
  ('us',): 2,
  ('about',): 3,
  ('death',): 1,
  ('?',): 1.0,
  ('short',): 3,
  ('philosophical',): 2,
  ('essay',): 2,
  ('p',): 2.0,
  ('=',): 1.0,
  ('np',): 4.0,
  ('we',): 17.0,
  ('claim',): 1,
  ('to',): 34.0,
  ('resolve',): 1,
  ('the',): 94.0,
  ('=?',): 1,
  ('problem',): 3,
  ('via',): 3,
  ('a',): 63.0,
  ('formal',): 3.0,
  ('argument',): 1,
  ('for',): 36.0,
  ('computational',): 1,
  ('geometry',): 1,
  ('column',): 1,
  ('38',): 1,
  ('recent',): 2,
  ('results',): 5.0,
  ('on',): 26.0,
  ('curve',): 1,
  ('reconstruction',): 3.0,
  ('are',): 8.0,
  ('described',): 2,
  ('weak',): 1,
  ('evolvability',): 2,
  ('equals',): 1,
  ('strong',): 1,
  ('an',): 10,
  ('updat

In [455]:
len([line.split(' ')[0] for line in dummy_lines])

100

In [458]:
len(list(dummy_lm.prob_kn_[2][('_UNK_',)].keys()))

78

In [423]:
dummy_lm = KneserNeyLanguageModel(dummy_lines, n=2)

In [436]:
dummy_lm = KneserNeyLanguageModel(dummy_lines, n=2)
print(sum([dummy_lm.get_next_token_prob('a', w_i) for w_i in dummy_lm.vocab]))
assert np.allclose(sum([dummy_lm.get_next_token_prob('a', w_i) for w_i in dummy_lm.vocab]), 1), "I told you not to break anything! :)"

19.0


AssertionError: I told you not to break anything! :)

In [397]:
sum([dummy_lm.get_next_token_prob('a', w_i) for w_i in dummy_lm.vocab])

0.0

In [459]:
#test that it's a valid probability model
for n in (1, 2, 3):
    dummy_lm = KneserNeyLanguageModel(dummy_lines, n=n)
    print(sum([dummy_lm.get_next_token_prob('a', w_i) for w_i in dummy_lm.vocab]))
    assert np.allclose(sum([dummy_lm.get_next_token_prob('a', w_i) for w_i in dummy_lm.vocab]), 1), "I told you not to break anything! :)"

0.999999999999999
19.0


AssertionError: I told you not to break anything! :)

In [348]:
for n in (1, 2, 3):
    lm = KneserNeyLanguageModel(train_lines, n=n, smoothing=<...>)
    ppx = perplexity(lm, test_lines)
    print("N = %i, Perplexity = %.5f" % (n, ppx))

<ipython-input-264-91263a57f5af>:21: RuntimeWarning: divide by zero encountered in log
  log_line_perplexity += max(np.log(lm.get_next_token_prob(prefix=' '.join(tokens[:i]), next_token=tokens[i])), min_logprob)


N = 1, Perplexity = 1832.23136
N = 2, Perplexity = 85653987.28543
N = 3, Perplexity = 61999196239911532363776.00000
